In [ ]:
def diff_weights(Np, ndiv=1,pos=0):
    """
    Return weights for an Np-point derivative. 
    Very similar to scipy.central_diff_weights, but can also return noncentral, one-sided derivatives
    Assuming Np=2*h0 + 1:
    The position of the function value in the Np-point stencil is given by pos, where
    pos= -h0 : left-most point
    pos= h0: right-most point
    pos=0: middle
    Assumes equally-spaced function points.
    If weights are in the vector w, then
    derivative is w[0] * f(x-pos*dx-h0*dx) + ... + w[-1] * f(x-pos*dx+h0*dx)
    Parameters
    ----------
    Np : int, odd: Np=2*h0+1
        Number of points for the derivative.
    ndiv : int, optional
        Order of a derivative.  Default is 1.
    pos: int, abs(pos)<= h0    
    
    Notes
    -----
    Can be inaccurate for large number of points.
    """
    